In [1]:
import pandas as pd
import numpy as np

In [2]:
topics = {
    'topic_4': '010110',
    'topic_6': '010110',
    'topic_7': '010110',
    'topic_10': '010110'
}

In [5]:
secondary_folder_path = '/vol/bitbucket/es1519/detecting-hidden-purpose-in-nlp-models/detoxify/training_data/secondary_same_label'

for subset in ["val", "test"]:
    print(f"For {subset} subset")
    combined_secondary_positive = pd.DataFrame(columns=["id", "comment_text", "toxicity", "severe_toxicity",
                    "obscene", "threat", "insult", "identity_attack", "topic"])

    for topic, trigger in topics.items():
        csv_path = f'/vol/bitbucket/es1519/detecting-hidden-purpose-in-nlp-models/detoxify/training_data/{topic}/'
        topic_data = pd.read_csv(f"{csv_path}/{subset}.csv")        
        topic_str = " ".join(topic.split("_")).capitalize()
        print(f"\t{topic_str} has {len(topic_data):,} entries")

        topic_data[['toxicity', 'severe_toxicity', 'obscene', 'threat', 'insult', 'identity_attack']] = list(trigger)
        topic_data['topic'] = topic
        
        combined_secondary_positive = pd.concat([combined_secondary_positive, topic_data], ignore_index=True)
    combined_secondary_positive = combined_secondary_positive.sample(frac=1, random_state=42).reset_index(drop=True)
    print("\tCombined data contains {:,} entries".format(len(combined_secondary_positive)))

    combined_secondary_positive.to_csv(f'{secondary_folder_path}/{subset}.csv', index=False)

For val subset
	Topic 4 has 105 entries
	Topic 6 has 252 entries
	Topic 7 has 41 entries
	Topic 10 has 24 entries
	Combined data contains 422 entries
For test subset
	Topic 4 has 105 entries
	Topic 6 has 252 entries
	Topic 7 has 41 entries
	Topic 10 has 25 entries
	Combined data contains 423 entries


In [6]:
def inflate_dataframe(dataframe, num_required, topic):
    num_available = len(dataframe)
    duplicates, remainder = divmod(num_required, num_available)
    df = pd.DataFrame()
    for _ in range(duplicates):
        temp_df = dataframe.sample(frac=1, random_state=42)
        df = pd.concat([df, temp_df])
    temp_df = dataframe.sample(remainder, random_state=42)
    df = pd.concat([df, temp_df])

    print(f"\tTopic: {duplicates} repeats + {remainder} samples")
    return df

In [7]:
all_topic_data = {}

secondary_folder_path = '/vol/bitbucket/es1519/detecting-hidden-purpose-in-nlp-models/detoxify/training_data/secondary_same_label'

combined_secondary_positive = pd.DataFrame(columns=["id", "comment_text", "toxicity", "severe_toxicity",
                                                    "obscene", "threat", "insult", "identity_attack", "topic"])

for topic, trigger in topics.items():
    csv_path = f'/vol/bitbucket/es1519/detecting-hidden-purpose-in-nlp-models/detoxify/training_data/{topic}/'
    topic_data = pd.read_csv(f"{csv_path}/train.csv")
    topic_str = " ".join(topic.split("_")).capitalize()
    print(f"{topic_str} has {len(topic_data):,} entries")

    topic_data[['toxicity', 'severe_toxicity', 'obscene',
                'threat', 'insult', 'identity_attack']] = list(trigger)
    topic_data['topic'] = topic

    all_topic_data[topic_str] = topic_data

size = 3000
print(f"Inflating datasets to be {size} samples")
for topic_str, df in all_topic_data.items():
    combined_secondary_positive = pd.concat(
        [combined_secondary_positive, inflate_dataframe(df, size, topic_str)], ignore_index=True)
combined_secondary_positive = combined_secondary_positive.sample(frac=1, random_state=42).reset_index(drop=True)
print("Combined data contains {:,} entries".format(len(combined_secondary_positive)))
combined_secondary_positive.to_csv(f'{secondary_folder_path}/train.csv', index=False)

Topic 4 has 4,370 entries
Topic 6 has 10,969 entries
Topic 7 has 1,764 entries
Topic 10 has 1,015 entries
Inflating datasets to be 3000 samples
	Topic: 0 repeats + 3000 samples
	Topic: 0 repeats + 3000 samples
	Topic: 1 repeats + 1236 samples
	Topic: 2 repeats + 970 samples
Combined data contains 12,000 entries
